In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import base64
#!pip install spotipy
import requests
from bs4 import BeautifulSoup
import time
from spotipy.oauth2 import SpotifyClientCredentials
!pip install spotipy --upgrade

In [2]:
charts_df = pd.read_csv('../data/charts.csv')
lyrics_df = pd.read_csv('../data/billboard_24years_lyrics_spotify.csv')
spotify_qualities_df = pd.read_csv('../data/Hot100.csv')

In [3]:
spotify_qualities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620 entries, 0 to 619
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track             620 non-null    object 
 1   Artist            620 non-null    object 
 2   Album             620 non-null    object 
 3   Year              620 non-null    int64  
 4   Duration          620 non-null    int64  
 5   Time_Signature    620 non-null    int64  
 6   Danceability      620 non-null    float64
 7   Energy            620 non-null    float64
 8   Key               620 non-null    int64  
 9   Loudness          620 non-null    float64
 10  Mode              620 non-null    int64  
 11  Speechiness       620 non-null    float64
 12  Acousticness      620 non-null    float64
 13  Instrumentalness  620 non-null    float64
 14  Liveness          620 non-null    float64
 15  Valence           620 non-null    float64
 16  Tempo             620 non-null    float64
 1

In [4]:
charts_df.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18


In [5]:
lyrics_df.head()

,ranking,song,band_singer,songurl,titletext,url,year,lyrics,uri,danceability,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,1,Breathe,Faith Hill,/wiki/Breathe_(Faith_Hill_song),Breathe,/wiki/Faith_Hill,2000,I can feel the magic floating in the air\nBein...,spotify:track:3y4LxiYMgDl4RethdzpmNe,0.529,...,0.000000,0.2510,0.278,136.859,audio_features,3y4LxiYMgDl4RethdzpmNe,https://api.spotify.com/v1/tracks/3y4LxiYMgDl4...,https://api.spotify.com/v1/audio-analysis/3y4L...,250547.0,4.0
1,2,Smooth,Santana,/wiki/Smooth_(Santana_song),Smooth,/wiki/Santana_(band),2000,"Man, it's a hot one\nLike seven inches from th...",spotify:track:0n2SEXB2qoRQg171q7XqeW,0.609,...,0.000005,0.2950,0.961,115.996,audio_features,0n2SEXB2qoRQg171q7XqeW,https://api.spotify.com/v1/tracks/0n2SEXB2qoRQ...,https://api.spotify.com/v1/audio-analysis/0n2S...,294987.0,4.0
2,2,Smooth,Rob Thomas,/wiki/Smooth_(Santana_song),Smooth,/wiki/Rob_Thomas_(musician),2000,"Man, it's a hot one\nLike seven inches from th...",spotify:track:5IALWUYK0zDSEmZgb4ICvc,0.590,...,0.807000,0.2990,0.724,115.983,audio_features,5IALWUYK0zDSEmZgb4ICvc,https://api.spotify.com/v1/tracks/5IALWUYK0zDS...,https://api.spotify.com/v1/audio-analysis/5IAL...,244924.0,4.0
3,3,Maria Maria,Santana,/wiki/Maria_Maria,Maria Maria,/wiki/Santana_(band),2000,"Ladies and gents, turn up your sound systems\n...",spotify:track:3XKIUb7HzIF1Vu9usunMzc,0.777,...,0.002010,0.0348,0.680,97.911,audio_features,3XKIUb7HzIF1Vu9usunMzc,https://api.spotify.com/v1/tracks/3XKIUb7HzIF1...,https://api.spotify.com/v1/audio-analysis/3XKI...,261973.0,4.0
4,3,Maria Maria,The Product G&B,/wiki/Maria_Maria,Maria Maria,/wiki/The_Product_G%26B,2000,Turn up this sound system\nTo the sound of Car...,spotify:track:3XKIUb7HzIF1Vu9usunMzc,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
client_id = 'f309e57f54944e3a916ac1076d26b76b'
client_secret = '647e3641f3e148df8e652f7b9bac5727'

# Encode credentials
auth_str = f"{client_id}:{client_secret}"
b64_auth_str = base64.b64encode(auth_str.encode()).decode()

# Request access token
auth_response = requests.post(
    'https://accounts.spotify.com/api/token',
    data={'grant_type': 'client_credentials'},
    headers={'Authorization': f'Basic {b64_auth_str}'}
)

# Extract token
access_token = auth_response.json()['access_token']
headers = {'Authorization': f'Bearer {access_token}'}

# Set up authentication
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret
))

for query in search_list:
    results = sp.search(q=query, type='track', limit=1)
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        print(f"Found: {track['name']} by {track['artists'][0]['name']}")
    else:
        print(f"No results for: {query}")

NameError: name 'spotipy' is not defined

In [38]:
results = sp.search(q='Shape of You Ed Sheeran', type='track', limit=1)
print(results['tracks']['items'][0]['name'])

NameError: name 'sp' is not defined

In [6]:
charts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330087 entries, 0 to 330086
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   date            330087 non-null  object 
 1   rank            330087 non-null  int64  
 2   song            330087 non-null  object 
 3   artist          330087 non-null  object 
 4   last-week       297775 non-null  float64
 5   peak-rank       330087 non-null  int64  
 6   weeks-on-board  330087 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 17.6+ MB


In [7]:
charts_df['artist'].value_counts()

artist
Taylor Swift                                      1023
Elton John                                         889
Madonna                                            857
Drake                                              787
Kenny Chesney                                      769
                                                  ... 
RiceGum Featuring Alissa Violet                      1
XXXTENTACION Featuring PnB Rock & Trippie Redd       1
Justin Bieber Featuring Burna Boy                    1
Justin Bieber Featuring BEAM                         1
Frankie Vaughan                                      1
Name: count, Length: 10205, dtype: int64

In [8]:
lyrics_df = lyrics_df.rename(columns={'ranking': 'rank'})
spotify_qualities_df = spotify_qualities_df.rename(columns={'Time_Signature': 'time_signature', 'Track': 'song', 'Year': 'year', 'Danceability': 'danceability', 'Energy': 'energy', 'Key': 'key', 'Loudness': 'loudness', 'Mode': 'mode', 'Speechiness': 'speechiness', 'Liveness': 'liveness', 'Valence': 'valence', 'Tempo': 'tempo'}) 
spotify_qualities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620 entries, 0 to 619
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song              620 non-null    object 
 1   Artist            620 non-null    object 
 2   Album             620 non-null    object 
 3   year              620 non-null    int64  
 4   Duration          620 non-null    int64  
 5   time_signature    620 non-null    int64  
 6   danceability      620 non-null    float64
 7   energy            620 non-null    float64
 8   key               620 non-null    int64  
 9   loudness          620 non-null    float64
 10  mode              620 non-null    int64  
 11  speechiness       620 non-null    float64
 12  Acousticness      620 non-null    float64
 13  Instrumentalness  620 non-null    float64
 14  liveness          620 non-null    float64
 15  valence           620 non-null    float64
 16  tempo             620 non-null    float64
 1

In [9]:
charts_df['datetime'] = pd.to_datetime(charts_df['date'])
charts_df['year'] = charts_df['datetime'].dt.year

In [10]:
charts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330087 entries, 0 to 330086
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            330087 non-null  object        
 1   rank            330087 non-null  int64         
 2   song            330087 non-null  object        
 3   artist          330087 non-null  object        
 4   last-week       297775 non-null  float64       
 5   peak-rank       330087 non-null  int64         
 6   weeks-on-board  330087 non-null  int64         
 7   datetime        330087 non-null  datetime64[ns]
 8   year            330087 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(3), object(3)
memory usage: 21.4+ MB


In [11]:
charts_and_lyrics_df = pd.merge(charts_df, lyrics_df, on=['year', 'song'], how='inner')
pd.set_option('display.max_columns', None)
charts_and_lyrics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55744 entries, 0 to 55743
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              55744 non-null  object        
 1   rank_x            55744 non-null  int64         
 2   song              55744 non-null  object        
 3   artist            55744 non-null  object        
 4   last-week         53799 non-null  float64       
 5   peak-rank         55744 non-null  int64         
 6   weeks-on-board    55744 non-null  int64         
 7   datetime          55744 non-null  datetime64[ns]
 8   year              55744 non-null  int32         
 9   rank_y            55744 non-null  int64         
 10  band_singer       55744 non-null  object        
 11  songurl           55744 non-null  object        
 12  titletext         55744 non-null  object        
 13  url               55744 non-null  object        
 14  lyrics            5574

In [27]:
range_charts_and_lyrics_df = charts_and_lyrics_df[
    (charts_and_lyrics_df['year'] >= 1999) &
    (charts_and_lyrics_df['year'] <= 2023)]
no_repeats_range_charts_and_lyrics_df = range_charts_and_lyrics_df.groupby('song').first().reset_index()
no_repeats_range_charts_and_lyrics_df.head(10)

,song,date,rank_x,artist,last-week,peak-rank,weeks-on-board,datetime,year,rank_y,band_singer,songurl,titletext,url,lyrics,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,featured_artist,main_artist
0,#Beautiful,2013-09-07,95,Mariah Carey Featuring Miguel,86.0,15,16,2013-09-07,2013,81,Mariah Carey,/wiki/Beautiful_(Mariah_Carey_song),#Beautiful,/wiki/Mariah_Carey,"Ah, ah, you’re beautiful (Yeah)\nAh, ah, you’r...",spotify:track:1ahAuX3F6tDdNRfMdiLsv6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN,Miguel,Mariah Carey
1,'03 Bonnie & Clyde,2003-03-29,46,Jay-Z Featuring Beyonce Knowles,37.0,4,23,2003-03-29,2003,37,Jay-Z,/wiki/%2703_Bonnie_%26_Clyde,'03 Bonnie & Clyde,/wiki/Jay-Z,"Uh-huh, uh, uh\nYou ready, B?\nLet's go get 'e...",spotify:track:5ljCWsDlSyJ41kwqym2ORw,0.759,0.678,9.0,-5.148,0.0,0.3140,0.230,0.0,0.1500,0.327,89.640,audio_features,5ljCWsDlSyJ41kwqym2ORw,https://api.spotify.com/v1/tracks/5ljCWsDlSyJ4...,https://api.spotify.com/v1/audio-analysis/5ljC...,205560.0,4.0,Beyonce Knowles,Jay-Z
2,1 Thing,2005-06-25,64,Amerie,61.0,8,20,2005-06-25,2005,67,Amerie,/wiki/1_Thing,1 Thing,/wiki/Amerie,"Woo!\nUh\nWoo!\nNa-na-na-na-na, oh (Uh, uh)\nN...",spotify:track:1mnqraQ8oV8MX92rdOFLWW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN,None,Amerie
3,"1, 2 Step",2005-07-23,50,Ciara Featuring Missy Elliott,44.0,2,39,2005-07-23,2005,5,Ciara,"/wiki/1,_2_Step","1, 2 Step",/wiki/Ciara,Welcome to the place I call home\nNew York Cit...,spotify:track:3x9LFpV6tzq0qWaBMvY2UT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN,Missy Elliott,Ciara
4,1-800-273-8255,2018-02-24,45,Logic Featuring Alessia Cara & Khalid,40.0,3,42,2018-02-24,2018,84,Logic,/wiki/1-800-273-8255_(song),1-800-273-8255,/wiki/Logic_(musician),"I've been on the low, I been taking my time\nI...",spotify:track:5tz69p7tJuGPeMGwNTxYuV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN,Alessia Cara & Khalid,Logic
5,"10,000 Hours",2020-05-09,46,Dan + Shay & Justin Bieber,41.0,4,30,2020-05-09,2020,23,Dan + Shay,"/wiki/10,000_Hours","10,000 Hours",/wiki/Dan_%2B_Shay,"Do you love the rain, does it make you dance\n...",spotify:track:2wrJq5XKLnmhRXHIAf9xBa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN,Justin Bieber,Dan + Shay
6,100 Years,2004-07-10,49,Five For Fighting,47.0,28,22,2004-07-10,2004,77,Five for Fighting,/wiki/100_Years_(song),100 Years,/wiki/Five_for_Fighting,I'm fifteen for a moment\nCaught in between te...,spotify:track:2lFlveK1y13WWp3vnQtrr3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN,None,Five For Fighting
7,19 Somethin',2003-04-05,46,Mark Wills,41.0,23,20,2003-04-05,2003,84,Mark Wills,/wiki/19_Somethin%27,19 Somethin',/wiki/Mark_Wills,Woo\nOh yeah\n\nI saw Star Wars at least eight...,spotify:track:0ElpjZyX8YBiXuYcDC1Ci9,0.580,0.816,4.0,-5.165,1.0,0.0726,0.132,0.0,0.0776,0.814,89.938,audio_features,0ElpjZyX8YBiXuYcDC1Ci9,https://api.spotify.com/v1/tracks/0ElpjZyX8YBi...,https://api.spotify.com/v1/audio-analysis/0Elp...,200467.0,4.0,None,Mark Wills
8,2 On,2014-11-01,43,Tinashe Featuring ScHoolboy Q,36.0,24,24,2014-11-01,2014,65,Tinashe,/wiki/2_On,2 On,/wiki/Tinashe,"Mustard on the beat, ho\n\nGive me all that yo...",spotify:track:3jVtllWS5CFFWLQng8sKsr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN,ScHoolboy Q,Tinashe
9,2 Phones,2016-07-09,45,Kevin Gates,42.0,17,25,2016-07-09,2016,57,Kevin Gates,/wiki/2_Phones,2 Phones,/wiki/Kevin_Gates,"Hello? What the deal? Bet\n\nI got two phones,...",spotify:track:0wdKiSBUT7aZkXUIdJWcwC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN,None,Kevin Gates


In [28]:
song_queries = no_repeats_range_charts_and_lyrics_df[['song', 'artist']].drop_duplicates()
search_list = (song_queries['song'] + ' ' + song_queries['artist']).tolist()

In [34]:
for query in search_list:
    results = sp.search(q=query, type='track', limit=1)
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        print(f"Found: {track['name']} by {track['artists'][0]['name']}")
    else:
        print(f"No results for: {query}")

NameError: name 'sp' is not defined

In [12]:
#charts_and_lyrics_qualities_df = pd.merge(charts_and_lyrics_df, spotify_qualities_df, on=['year', 'song', 'time_signature', 'danceability', 'energy', 'loudness', 'speechiness', 'liveness', 'valence', 'tempo'], how='inner')
#pd.set_option('display.max_columns', None)
#charts_and_lyrics_qualities_df= charts_and_lyrics_qualities_df['song'].nunique()
#charts_and_lyrics_qualities_df.head(30)

In [13]:
artists = sorted(charts_and_lyrics_df['artist'].astype(str).unique().tolist())
artists

["'N Sync",
 "'N Sync Featuring Nelly",
 '112',
 '112 Featuring Foxy Brown',
 '2 Chainz',
 '2 Chainz Featuring Drake',
 '2 Pistols Featuring T-Pain & Tay Dizm',
 '21 Savage',
 '24kGoldn Featuring iann dior',
 '2Pac',
 '3 Doors Down',
 '3LW',
 '3OH!3',
 '3OH!3 Featuring Ke$ha',
 '5 Seconds Of Summer',
 '50 Cent',
 '50 Cent Featuring Justin Timberlake & Timbaland',
 '50 Cent Featuring Nate Dogg',
 '50 Cent Featuring Olivia',
 '6ix9ine',
 '98 Degrees',
 'A Boogie Wit da Hoodie',
 'A Boogie Wit da Hoodie Featuring 6ix9ine',
 'A Boogie Wit da Hoodie Featuring Kodak Black',
 'A Great Big World & Christina Aguilera',
 'A$AP Ferg Featuring Nicki Minaj',
 'AJR',
 'AWOLNATION',
 'Aaliyah',
 'Ace Hood Featuring Future & Rick Ross',
 'Adele',
 'Aerosmith',
 'Akon',
 "Akon Featuring Colby O'Donis & Kardinal Offishall",
 'Akon Featuring Eminem',
 'Akon Featuring Snoop Dogg',
 'Akon Featuring Styles P.',
 'Alan Jackson',
 'Alan Jackson & Jimmy Buffett',
 'Alanis Morissette',
 'Alessia Cara',
 'Alex C

In [14]:
featured_artists= ['Featuring', '&', 'With']
def extract_featured(artist):
    for featured in featured_artists:
          if featured in artist:
            parts = artist.split(featured)
            if len(parts) > 1:
                return parts[1].strip()
    return None

def extract_main_artist(artist):
    for featured in featured_artists:
        if featured in artist:
            parts = artist.split(featured)
            return parts[0].strip()
    return artist.strip()

charts_and_lyrics_df['featured_artist'] = charts_and_lyrics_df['artist'].apply(extract_featured)
charts_and_lyrics_df['main_artist'] = charts_and_lyrics_df['artist'].apply(extract_main_artist)
charts_and_lyrics_df['main_artist'].head(20)

0      The Kid LAROI
1      The Kid LAROI
2          Lil Nas X
3          Lil Nas X
4       Walker Hayes
5         Ed Sheeran
6              Drake
7              Drake
8              Drake
9     Olivia Rodrigo
10          Dua Lipa
11            Wizkid
12            Wizkid
13            Wizkid
14          Doja Cat
15          Doja Cat
16     Glass Animals
17          Maneskin
18          Doja Cat
19          Doja Cat
Name: main_artist, dtype: object

In [15]:
spotify_qualities_df.dtypes

song                 object
Artist               object
Album                object
year                  int64
Duration              int64
time_signature        int64
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
Acousticness        float64
Instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
Popularity            int64
dtype: object

In [19]:
#lyrics_df['time_signature'].value_counts()
spotify_qualities_df['time_signature'].value_counts()

time_signature
4    562
3     51
5      4
1      3
Name: count, dtype: int64

In [21]:
charts_and_lyrics_df[charts_and_lyrics_df['time_signature'] == 5]

,date,rank_x,song,artist,last-week,peak-rank,weeks-on-board,datetime,year,rank_y,band_singer,songurl,titletext,url,lyrics,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,featured_artist,main_artist
44445,2004-12-25,46,Lean Back,Terror Squad,37.0,1,27,2004-12-25,2004,10,Fat Joe,/wiki/Lean_Back,Lean Back,/wiki/Fat_Joe,"Feel this shit right here, Scott Storch, nigga...",spotify:track:2uiEqPjMmIJXOKplzsEASP,0.611,0.754,1.0,-5.035,0.0,0.366,0.116,0.00000,0.0945,0.5650,126.961,audio_features,2uiEqPjMmIJXOKplzsEASP,https://api.spotify.com/v1/tracks/2uiEqPjMmIJX...,https://api.spotify.com/v1/audio-analysis/2uiE...,290267.0,5.0,None,Terror Squad
44460,2004-12-18,37,Lean Back,Terror Squad,24.0,1,26,2004-12-18,2004,10,Fat Joe,/wiki/Lean_Back,Lean Back,/wiki/Fat_Joe,"Feel this shit right here, Scott Storch, nigga...",spotify:track:2uiEqPjMmIJXOKplzsEASP,0.611,0.754,1.0,-5.035,0.0,0.366,0.116,0.00000,0.0945,0.5650,126.961,audio_features,2uiEqPjMmIJXOKplzsEASP,https://api.spotify.com/v1/tracks/2uiEqPjMmIJX...,https://api.spotify.com/v1/audio-analysis/2uiE...,290267.0,5.0,None,Terror Squad
44478,2004-12-11,24,Lean Back,Terror Squad,26.0,1,25,2004-12-11,2004,10,Fat Joe,/wiki/Lean_Back,Lean Back,/wiki/Fat_Joe,"Feel this shit right here, Scott Storch, nigga...",spotify:track:2uiEqPjMmIJXOKplzsEASP,0.611,0.754,1.0,-5.035,0.0,0.366,0.116,0.00000,0.0945,0.5650,126.961,audio_features,2uiEqPjMmIJXOKplzsEASP,https://api.spotify.com/v1/tracks/2uiEqPjMmIJX...,https://api.spotify.com/v1/audio-analysis/2uiE...,290267.0,5.0,None,Terror Squad
44498,2004-12-04,26,Lean Back,Terror Squad,18.0,1,24,2004-12-04,2004,10,Fat Joe,/wiki/Lean_Back,Lean Back,/wiki/Fat_Joe,"Feel this shit right here, Scott Storch, nigga...",spotify:track:2uiEqPjMmIJXOKplzsEASP,0.611,0.754,1.0,-5.035,0.0,0.366,0.116,0.00000,0.0945,0.5650,126.961,audio_features,2uiEqPjMmIJXOKplzsEASP,https://api.spotify.com/v1/tracks/2uiEqPjMmIJX...,https://api.spotify.com/v1/audio-analysis/2uiE...,290267.0,5.0,None,Terror Squad
44524,2004-11-27,18,Lean Back,Terror Squad,13.0,1,23,2004-11-27,2004,10,Fat Joe,/wiki/Lean_Back,Lean Back,/wiki/Fat_Joe,"Feel this shit right here, Scott Storch, nigga...",spotify:track:2uiEqPjMmIJXOKplzsEASP,0.611,0.754,1.0,-5.035,0.0,0.366,0.116,0.00000,0.0945,0.5650,126.961,audio_features,2uiEqPjMmIJXOKplzsEASP,https://api.spotify.com/v1/tracks/2uiEqPjMmIJX...,https://api.spotify.com/v1/audio-analysis/2uiE...,290267.0,5.0,None,Terror Squad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53536,2001-02-03,19,Angel,Shaggy Featuring Rayvon,24.0,19,6,2001-02-03,2001,17,Rayvon,/wiki/Angel_(Shaggy_song),Angel,/wiki/Rayvon,Now this one's dedicated to all the youths who...,spotify:track:2Ei4V1qAiO2B1s4xtl7cl9,0.238,0.119,10.0,-13.600,0.0,0.038,0.955,0.00133,0.1140,0.0626,80.639,audio_features,2Ei4V1qAiO2B1s4xtl7cl9,https://api.spotify.com/v1/tracks/2Ei4V1qAiO2B...,https://api.spotify.com/v1/audio-analysis/2Ei4...,225595.0,5.0,Rayvon,Shaggy
53579,2001-01-27,24,Angel,Shaggy Featuring Rayvon,39.0,24,5,2001-01-27,2001,17,Rayvon,/wiki/Angel_(Shaggy_song),Angel,/wiki/Rayvon,Now this one's dedicated to all the youths who...,spotify:track:2Ei4V1qAiO2B1s4xtl7cl9,0.238,0.119,10.0,-13.600,0.0,0.038,0.955,0.00133,0.1140,0.0626,80.639,audio_features,2Ei4V1qAiO2B1s4xtl7cl9,https://api.spotify.com/v1/tracks/2Ei4V1qAiO2B...,https://api.spotify.com/v1/audio-analysis/2Ei4...,225595.0,5.0,Rayvon,Shaggy
53621,2001-01-20,39,Angel,Shaggy Featuring Rayvon,56.0,39,4,2001-01-20,2001,17,Rayvon,/wiki/Angel_(Shaggy_song),Angel,/wiki/Rayvon,Now this one's dedicated to all the youths who...,spotify:track:2Ei4V1qAiO2B1s4xtl7cl9,0.238,0.119,10.0,-13.600,0.0,0.038,0.955,0.00133,0.1140,0.0626,80.639,audio_features,2Ei4V1qAiO2B1s4xtl7cl9,https://api.spotify.com/v1/tracks/2Ei4V1qAiO2B...,https://api.spotify.com

In [18]:
spotify_qualities_df[spotify_qualities_df['time_signature'] == 5.0]

,song,Artist,Album,year,Duration,time_signature,danceability,energy,key,loudness,mode,speechiness,Acousticness,Instrumentalness,liveness,valence,tempo,Popularity
243,this is what falling in love feels like,JVKE,this is what ____ feels like (Vol. 1-4),2022,120308,5,0.422,0.440,11,-5.530,1,0.0544,0.617,0.000000,0.0837,0.332,128.934,76
410,Chun-Li,Nicki Minaj,Queen,2018,191600,5,0.686,0.730,6,-5.077,0,0.3740,0.278,0.000048,0.1060,0.474,128.943,66
487,Ginger Me,Rema,Ginger Me,2023,205892,5,0.567,0.577,8,-7.833,0,0.1340,0.134,0.000397,0.1450,0.719,200.008,49
559,I Hate U,SZA,SOS,2021,173906,5,0.533,0.417,7,-8.665,0,0.1480,0.514,0.000000,0.1140,0.376,106.534,70


In [ ]:
#pip install billboard.py

url = 'https://www.billboard.com/charts/hot-100/'
headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

titles = [tag.get_text(strip=True) for tag in soup.select('li ul li h3')]
artists = [tag.get_text(strip=True) for tag in soup.select('li ul li span.c-label') if not tag.attrs.get('class', [])[-1].startswith('c-label--chart')]

# Clean and zip
hot_100_songs = list(zip(titles[:100], artists[:100]))  # top 100 only

# Preview
for i, (title, artist) in enumerate(hot_100_songs[:5], start=1):
    print(f"{i}. {title} by {artist}")